In [1]:
import numpy as np

def modified_gram_schmidt(A):
    """
    Perform QR decomposition using the Modified Gram-Schmidt algorithm.
    
    Parameters:
    A : numpy.ndarray
        The input matrix of shape (m, n).
        
    Returns:
    Q : numpy.ndarray
        An orthogonal matrix of shape (m, n).
    R : numpy.ndarray
        An upper triangular matrix of shape (n, n).
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    V = A.copy()
    
    for i in range(n):
        R[i, i] = np.linalg.norm(V[:, i])
        if R[i, i] == 0:
            raise ValueError("Matrix is rank deficient.")
        Q[:, i] = V[:, i] / R[i, i]
        for j in range(i+1, n):
            R[i, j] = np.dot(Q[:, i], V[:, j])
            V[:, j] = V[:, j] - R[i, j] * Q[:, i]
            
    return Q, R

def householder_reflection(a):
    """
    Compute the Householder vector for vector a.
    Returns vector v such that (I - 2*v*v^T) * a = alpha * e1
    """
    v = a.copy()
    alpha = -np.sign(a[0]) * np.linalg.norm(a)
    v[0] = v[0] - alpha
    v = v / np.linalg.norm(v)
    return v

def bidiagonalization(A):
    """
    Reduce matrix A to bidiagonal form B using Householder transformations.
    
    Parameters:
    A : numpy.ndarray
        The input matrix of shape (m, n).
        
    Returns:
    B : numpy.ndarray
        The bidiagonal matrix.
    U : numpy.ndarray
        The left orthogonal matrix.
    V : numpy.ndarray
        The right orthogonal matrix.
    """
    m, n = A.shape
    U = np.eye(m)
    V = np.eye(n)
    B = A.copy()
    
    for k in range(min(m, n)):
        # Left Householder reflection
        x = B[k:, k]
        v = householder_reflection(x)
        B[k:, :] -= 2 * np.outer(v, np.dot(v, B[k:, :]))
        U_k = np.eye(m)
        U_k[k:, k:] -= 2 * np.outer(v, v)
        U = np.dot(U, U_k)
        
        if k < n - 1:
            # Right Householder reflection
            x = B[k, k+1:]
            v = householder_reflection(x)
            B[:, k+1:] -= 2 * np.outer(np.dot(B[:, k+1:], v), v)
            V_k = np.eye(n)
            V_k[k+1:, k+1:] -= 2 * np.outer(v, v)
            V = np.dot(V, V_k)
    
    return B, U, V

def golub_reinsch_svd(A, tol=1e-10, maxiter=1000):
    # Step 1: Bidiagonalization
    B, U, V = bidiagonalization(A)
    
    # Step 2: Diagonalization of bidiagonal matrix B
    m, n = B.shape
    S = np.zeros(min(m, n))
    for _ in range(maxiter):
        off_diagonal = np.diag(B, k=1)
        if np.all(np.abs(off_diagonal) < tol):
            break
        # For simplicity, use numpy's SVD on B
        U_bi, S_bi, Vt_bi = np.linalg.svd(B, full_matrices=False)
        U = np.dot(U, U_bi)
        V = np.dot(V, Vt_bi.T)
        S = S_bi
        break  # Simplified implementation
    
    return U, S, V.T

# Example usage:
if __name__ == "__main__":
    # Test the modified Gram-Schmidt algorithm
    A = np.array([[1, 1, 0],
                  [1, 0, 1],
                  [0, 1, 1]], dtype=float)
    A = np.random.rand(180, 180)
    Q, R = modified_gram_schmidt(A)
    print("Q from Modified Gram-Schmidt:")
    print(Q)
    print("R from Modified Gram-Schmidt:")
    print(R)
    print("Reconstructed A (Q @ R):")
    print(np.dot(Q, R))
    
    # Test the Golub-Reinsch SVD algorithm
    U, S, Vt = golub_reinsch_svd(A)
    print("\nLeft singular vectors U:")
    print(U)
    print("Singular values S:")
    print(S)
    print("Right singular vectors V^T:")
    print(Vt)
    print("Reconstructed A (U @ diag(S) @ V^T):")
    print(np.dot(U, np.dot(np.diag(S), Vt)))


Q from Modified Gram-Schmidt:
[[ 0.11906566  0.05130548 -0.02762944 ... -0.01541611  0.14751652
   0.05309647]
 [ 0.08194041 -0.06505713  0.03905157 ...  0.13294644  0.06949415
   0.07504348]
 [ 0.08974042  0.03270935 -0.09068294 ...  0.11343594  0.02214502
  -0.05559539]
 ...
 [ 0.09030095 -0.00427031 -0.03807986 ...  0.06888627 -0.00997955
   0.04990121]
 [ 0.08194986  0.03266135  0.08686291 ... -0.13471544 -0.07491663
  -0.1095077 ]
 [ 0.04722443  0.08675826  0.1007071  ...  0.03521782 -0.07471692
   0.02784937]]
R from Modified Gram-Schmidt:
[[ 7.50177964  5.51712238  6.08901442 ...  5.92565269  5.55973548
   6.21598329]
 [ 0.          5.24231647  2.23372286 ...  1.98206815  2.03867805
   1.69541333]
 [ 0.          0.          4.87744128 ...  1.64376891  1.3760606
   1.6407848 ]
 ...
 [ 0.          0.          0.         ...  0.3023325  -0.22953168
   0.36362067]
 [ 0.          0.          0.         ...  0.          0.53666448
  -0.10578823]
 [ 0.          0.          0.         .

In [2]:
import numpy as np
import matplotlib as plt
def modified_gram_schmidt(A):
    """
    Perform QR decomposition using the Modified Gram-Schmidt algorithm.
    
    Parameters:
    A : numpy.ndarray
        The input matrix of shape (m, n).
        
    Returns:
    Q : numpy.ndarray
        An orthogonal matrix of shape (m, n).
    R : numpy.ndarray
        An upper triangular matrix of shape (n, n).
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    V = A.copy()
    
    for i in range(n):
        R[i, i] = np.linalg.norm(V[:, i])
        if R[i, i] == 0:
            raise ValueError("Matrix is rank deficient.")
        Q[:, i] = V[:, i] / R[i, i]
        for j in range(i+1, n):
            R[i, j] = np.dot(Q[:, i], V[:, j])
            V[:, j] = V[:, j] - R[i, j] * Q[:, i]
            
    return Q, R

def householder_reflection(a):
    """
    Compute the Householder vector for vector a.
    Returns vector v such that (I - 2*v*v^T) * a = alpha * e1
    """
    v = a.copy()
    norm_a = np.linalg.norm(a)
    if norm_a == 0:
        # The vector is zero; no reflection is needed.
        v = np.zeros_like(a)
        v[0] = 1.0  # Set the first element to 1 to define a valid reflector.
        return v
    else:
        alpha = -np.sign(a[0]) * norm_a
        v[0] = v[0] - alpha
        norm_v = np.linalg.norm(v)
        if norm_v == 0:
            # v is zero after the adjustment; set to standard basis vector.
            v = np.zeros_like(a)
            v[0] = 1.0
            return v
        else:
            v = v / norm_v
            return v


def bidiagonalization(A):
    """
    Reduce matrix A to bidiagonal form B using Householder transformations.
    
    Parameters:
    A : numpy.ndarray
        The input matrix of shape (m, n).
        
    Returns:
    B : numpy.ndarray
        The bidiagonal matrix.
    U : numpy.ndarray
        The left orthogonal matrix.
    V : numpy.ndarray
        The right orthogonal matrix.
    """
    m, n = A.shape
    U = np.eye(m)
    V = np.eye(n)
    B = A.copy()
    
    for k in range(min(m, n)):
        # Left Householder reflection
        x = B[k:, k]
        v = householder_reflection(x)
        B[k:, :] -= 2 * np.outer(v, np.dot(v, B[k:, :]))
        U_k = np.eye(m)
        U_k[k:, k:] -= 2 * np.outer(v, v)
        U = np.dot(U, U_k)
        
        if k < n - 1:
            # Right Householder reflection
            x = B[k, k+1:]
            v = householder_reflection(x)
            B[:, k+1:] -= 2 * np.outer(np.dot(B[:, k+1:], v), v)
            V_k = np.eye(n)
            V_k[k+1:, k+1:] -= 2 * np.outer(v, v)
            V = np.dot(V, V_k)
        
    return B, U, V

def golub_reinsch_svd(A, tol=1e-10, maxiter=1000):
    # Step 1: Bidiagonalization
    B, U, V = bidiagonalization(A)
    
    # Step 2: Diagonalization of bidiagonal matrix B
    m, n = B.shape
    S = np.zeros(min(m, n))
    # For simplicity, use numpy's SVD on B
    U_bi, S_bi, Vt_bi = np.linalg.svd(B, full_matrices=False)
    U = np.dot(U, U_bi)
    V = np.dot(V, Vt_bi.T)
    S = S_bi
    return U, S, V.T

def svd_small(A):
    U, S, Vt = golub_reinsch_svd(A)
    return U, S, Vt

def qr_small(A):
    
    Q, R = modified_gram_schmidt(A)
    return Q, R

def streaming_svd(data_generator, k=None):
   
    U = None
    S = None
    Vt = None
    total_m = 0  # Total number of samples processed
    for batch in data_generator:
        n, m_new = batch.shape  # batch shape (n x m_new)
        if U is None:
            # First batch, compute SVD directly
            U, S, Vt = golub_reinsch_svd(batch)
            if k is not None:
                U = U[:, :k]
                S = S[:k]
                Vt = Vt[:k, :]
            total_m += m_new
        else:
            # Update SVD with new batch
            C = batch  # shape (n x m_new)
            # Step 1: Compute projections P = U^T C
            P = np.dot(U.T, C)  # shape (k x m_new)
            # Step 2: Compute residual R = C - U P
            R = C - np.dot(U, P)  # shape (n x m_new)
            # Step 3: Compute QR decomposition of R
            Q_r, R_r = qr_small(R)  # Q_r: (n x k_r), R_r: (k_r x m_new)
            # Step 4: Form K matrix
            S_matrix = np.diag(S)  # shape (k x k)
            upper = np.hstack((S_matrix, P))  # shape (k x (k + m_new))
            lower = np.hstack((np.zeros((R_r.shape[0], S_matrix.shape[1])), R_r))  # shape (k_r x (k + m_new))
            K = np.vstack((upper, lower))  # shape ((k + k_r) x (k + m_new))
            # Step 5: Compute SVD of K
            U_k, S_k, Vt_k = svd_small(K)
            # Step 6: Update U
            U_combined = np.hstack((U, Q_r))  # shape (n x (k + k_r))
            U_new = np.dot(U_combined, U_k)  # shape (n x r_new)
            # Step 7: Update Vt
            # Expand Vt with zeros to match dimensions
            zeros_Vt = np.zeros((Vt.shape[0], m_new))  # shape (k x m_new)
            Vt_expanded = np.hstack((Vt, zeros_Vt))  # shape (k x (total_m + m_new))
            zeros_identity = np.zeros((R_r.shape[0], total_m))  # shape (k_r x total_m)
            identity_R = np.eye(R_r.shape[0])  # shape (k_r x k_r)
            Vt_new_bottom = np.hstack((zeros_identity, identity_R))  # shape (k_r x (total_m + m_new))
            Vt_combined = np.vstack((Vt_expanded, Vt_new_bottom))  # shape ((k + k_r) x (total_m + m_new))
            Vt_new = np.dot(Vt_k, Vt_combined)  # shape (r_new x (total_m + m_new))
            # Step 8: Truncate if necessary
            if k is not None:
                U_new = U_new[:, :k]
                S_k = S_k[:k]
                Vt_new = Vt_new[:k, :]
            U = U_new
            S = S_k
            Vt = Vt_new
            total_m += m_new
    return U, S, Vt

# Example usage (assuming data_generator yields batches of MNIST data)
def data_generator():
    # For demonstration, let's generate random data batches
    n = 784  # Number of features (e.g., flattened 28x28 images)
    batch_size = 50
    num_batches = 10
    for _ in range(num_batches):
        batch = np.random.randn(n, batch_size)
        yield batch
from sklearn.datasets import fetch_openml


# Step 2: Create data generator
def data_generator_mnist(X, batch_size=50):
    n_samples = X.shape[0]
    num_batches = n_samples // batch_size
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size].T  # Shape (784, batch_size)
        yield batch
# Fetch MNIST data
from sklearn.datasets import fetch_openml

# Fetch MNIST data
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

# Ensure that data is in NumPy array format
X = mnist.data.astype(np.float64)  # Ensure data is float64
y = mnist.target.astype(np.int64)

# Normalize the data
X = X / 255.0
print("Data shape:", X.shape)
print("Data type:", X.dtype)
print("Sample labels:", y[:5])

# Step 2: Create data generator
def data_generator_mnist(X, batch_size=50):
    n_samples = X.shape[0]
    num_batches = n_samples // batch_size
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size].T  # Shape (784, batch_size)
        yield batch

# Step 3: Perform streaming SVD
batch_size = 50
k = 10  # Number of singular values/vectors to keep
gen = data_generator_mnist(X, batch_size=50)
batch = next(gen)
print("Batch shape:", batch.shape)
print("Batch data type:", batch.dtype)
U_batch, S_batch, Vt_batch = golub_reinsch_svd(batch)
print("U shape:", U_batch.shape)
print("S shape:", S_batch.shape)
print("Vt shape:", Vt_batch.shape)

U, S, Vt = streaming_svd(data_generator_mnist(X, batch_size), k=k)

# Step 4: Visualize the results (optional)
# Plot singular values
plt.figure(figsize=(8, 5))
plt.plot(S, 'o-', linewidth=2)
plt.title('Singular Values of MNIST Dataset')
plt.xlabel('Index')
plt.ylabel('Singular Value')
plt.grid(True)
plt.show()

# Display the top singular vectors as images
num_components_to_display = 10
fig, axes = plt.subplots(1, num_components_to_display, figsize=(15, 5))
for i in range(num_components_to_display):
    singular_vector = U[:, i]
    image = singular_vector.reshape(28, 28)
    axes[i].imshow(image, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'Component {i+1}')
plt.show()


Data shape: (70000, 784)
Data type: float64
Sample labels: [5 0 4 1 9]
Batch shape: (784, 50)
Batch data type: float64
U shape: (784, 50)
S shape: (50,)
Vt shape: (50, 50)


AttributeError: module 'matplotlib' has no attribute 'figure'